In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('../data/train.csv')

In [3]:
train.head()

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating
0,259,255,4,874724710,My Best Friend's Wedding (1997),Romance,4.0
1,259,286,4,874724727,"English Patient, The (1996)","Romance, War",4.0
2,259,298,4,874724754,Face/Off (1997),"Action, Sci-Fi, Thriller",4.0
3,259,185,4,874724781,Psycho (1960),"Horror, Romance, Thriller",4.0
4,259,173,4,874724843,"Princess Bride, The (1987)","Action, Adventure, Romance",4.0


In [25]:
train.groupby('user_id')['movie_title'].apply(list).apply(lambda x: len(x)).describe()

count    621.000000
mean     102.394525
std       94.180971
min        1.000000
25%       35.000000
50%       65.000000
75%      146.000000
max      737.000000
Name: movie_title, dtype: float64

In [26]:
train

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating
0,259,255,4,874724710,My Best Friend's Wedding (1997),Romance,4.0
1,259,286,4,874724727,"English Patient, The (1996)","Romance, War",4.0
2,259,298,4,874724754,Face/Off (1997),"Action, Sci-Fi, Thriller",4.0
3,259,185,4,874724781,Psycho (1960),"Horror, Romance, Thriller",4.0
4,259,173,4,874724843,"Princess Bride, The (1987)","Action, Adventure, Romance",4.0
...,...,...,...,...,...,...,...
63582,642,8,5,885603662,Babe (1995),Children,5.0
63583,642,234,1,885603662,Jaws (1975),"Action, Horror",1.0
63584,642,969,2,885603662,Winnie the Pooh and the Blustery Day (1968),"Animation, Children",2.0
63585,642,181,5,885603699,Return of the Jedi (1983),"Action, Adventure, Romance, Sci-Fi, War",5.0


In [318]:
user_movies = train.groupby('user_id').apply(lambda x: x['movie_title'].tolist())

In [290]:
users_top_movies = train.groupby('user_id').apply(lambda x: x.sort_values('rating', ascending=False).head(10)['movie_title'].to_list())

In [291]:
user_ratings = train.drop_duplicates(['user_id', 'movie_title']).pivot(index='user_id', columns='movie_title', values='rating').fillna(0)

In [297]:
from scipy.sparse import csr_matrix
sparse_user_ratings = csr_matrix(user_ratings)

In [292]:
ten_closest_users_by_rating_distance = user_ratings.apply(
    lambda x: user_ratings.loc[user_ratings.index != x.name]
    .apply(lambda y: ((y - x) ** 2).sum(), axis=1)
    .sort_values()
    .head(10)
    .index.to_list(),
    axis=1,
)

In [294]:
ten_closest_users_by_rating_distance.head()

user_id
1      [933, 521, 715, 738, 868, 44, 538, 77, 198, 508]
5     [638, 275, 723, 700, 746, 55, 517, 231, 483, 563]
6     [321, 452, 123, 929, 553, 237, 338, 82, 492, 745]
8       [55, 153, 352, 37, 700, 517, 785, 80, 638, 746]
10      [321, 6, 123, 237, 426, 710, 908, 397, 398, 82]
dtype: object

In [302]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [312]:
ten_closest_users_by_cosine_similarity = pd.DataFrame(cosine_similarity(sparse_user_ratings) - np.identity(sparse_user_ratings.shape[0]), index = user_ratings.index, columns = user_ratings.index).apply(lambda x: list(x.sort_values(ascending = False).head(10).index), axis=1)

In [315]:
ten_closest_users_by_cosine_similarity.head()

user_id
1     [916, 457, 268, 435, 429, 823, 301, 276, 889, ...
5      [648, 407, 307, 497, 268, 276, 22, 622, 804, 70]
6       [18, 194, 716, 10, 666, 151, 321, 85, 389, 854]
8      [746, 158, 37, 352, 638, 425, 22, 627, 671, 538]
10      [6, 406, 666, 389, 321, 398, 524, 716, 194, 18]
dtype: object

In [363]:
new_line = '\n'
for each in ten_closest_users_by_rating_distance.iloc[:1].index:
    Input = f"""I am user {each}.  The users who are most like me are {ten_closest_users_by_rating_distance[each]}.
The top movies for each of these users are:
{new_line.join(pd.Series(users_top_movies.loc[ten_closest_users_by_rating_distance[each]].index).apply(lambda x: ": ".join([str(x), ", ".join(users_top_movies.loc[x])])).to_list())}.
Please recommend ten movies for me to watch that I have not seen and provide brackets around the movies and their years so I can easily parse them."""
    

In [434]:
new_line = '\n'

def createPrompt(each):
    return f"""I am user {each}.
The most recent ten movies I have seen are are:
{", ".join(user_movies[each][-10:])}.
The users who are most like me are {ten_closest_users_by_rating_distance[each]}.
The top movies for each of these users are:
{new_line.join(pd.Series(users_top_movies.loc[ten_closest_users_by_rating_distance[each]].index).apply(lambda x: ": ".join([str(x), ", ".join(users_top_movies.loc[x])])).to_list())}.
Please recommend ten movies for me to watch that I have not seen and provide brackets around the movies and their years so I can easily parse them."""
    

In [371]:
Input

"I am user 1.\nThe movies I have seen are are:\nEmpire Strikes Back, The (1980), Monty Python and the Holy Grail (1974), Jean de Florette (1986), Reservoir Dogs (1992), Manon of the Spring (Manon des sources) (1986), Dead Poets Society (1989), Godfather: Part II, The (1974), Postino, Il (1994), Godfather, The (1972), Fifth Element, The (1997), Return of the Jedi (1983), Rock, The (1996), Mystery Science Theater 3000: The Movie (1996), Toy Story (1995), Chasing Amy (1997), Star Wars (1977), Grosse Pointe Blank (1997), Men in Black (1997), Austin Powers: International Man of Mystery (1997), Pillow Book, The (1995), In the Company of Men (1997), Welcome to the Dollhouse (1995), Lone Star (1996), Ridicule (1996), Antonia's Line (1995), Frighteners, The (1996), Big Night (1996), Unhook the Stars (1996), Twelve Monkeys (1995), Mars Attacks! (1996), Mr. Holland's Opus (1995), Devil's Own, The (1997), Mimic (1997), Spitfire Grill, The (1996), Rumble in the Bronx (1995), Event Horizon (1997), J

In [258]:
from openai import OpenAI
from key import OPENAI_API_KEY
client = OpenAI(api_key=OPENAI_API_KEY)

In [372]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages= [{ 'role':'user','content' : Input}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        )

In [392]:
createPrompt(ten_closest_users_by_rating_distance.iloc[:1].index[0])

"I am user 1.  The users who are most like me are [933, 521, 715, 738, 868, 44, 538, 77, 198, 508].\nThe top movies for each of these users are:\n933: Silence of the Lambs, The (1991), Pulp Fiction (1994), Fargo (1996), Apocalypse Now (1979), Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963), Godfather, The (1972), Clockwork Orange, A (1971), Braveheart (1995), Shawshank Redemption, The (1994), Sting, The (1973)\n521: Mystery Science Theater 3000: The Movie (1996), Clerks (1994), Sneakers (1992), Chasing Amy (1997), Akira (1988), Usual Suspects, The (1995), Amadeus (1984), Happy Gilmore (1996), Clockwork Orange, A (1971), Pulp Fiction (1994)\n715: Schindler's List (1993), Patton (1970), Shawshank Redemption, The (1994), Apollo 13 (1995), Die Hard (1988), Groundhog Day (1993), Pulp Fiction (1994), North by Northwest (1959), Aliens (1986), Silence of the Lambs, The (1991)\n738: Star Trek: The Wrath of Khan (1982), Raiders of the Lost Ark (1981), Princess Bride, 

In [435]:
test = pd.Series(ten_closest_users_by_rating_distance.index, index = ten_closest_users_by_rating_distance.index).loc[user_movies.index.intersection(user_movies_test.index)[:5]].apply(lambda x: client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages= [{ 'role':'user','content' : createPrompt(x)}],
        # temperature=0,
        # max_tokens=512,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
        ))

In [436]:
recommendations = test.apply(lambda x: [
    each.split("]")[0]
    for each in ([
        each
        for each in x.choices[0].message.content.split("[")
    ][1:])
])

In [437]:
recommendations

user_id
1     [Blade Runner, Pulp Fiction, Shawshank Redempt...
13    [Fight Club (1999), Eternal Sunshine of the Sp...
14    [Pulp Fiction (1994), Fight Club (1999), Etern...
23    [The Shawshank Redemption (1994), Pulp Fiction...
30    [Pulp Fiction (1994), The Shawshank Redemption...
Name: user_id, dtype: object

In [323]:
response

ChatCompletion(id='chatcmpl-8TZFXnptntBJSlpD10pwyQHp3nlsg', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Here are ten movie recommendations for you to watch:\n\n1. [Pulp Fiction (1994)] - Directed by Quentin Tarantino, this iconic crime film intertwines various characters and their stories in a non-linear narrative.\n2. [The Shawshank Redemption (1994)] - A gripping drama about a man who, despite being wrongfully convicted, finds hope and redemption during his time in prison.\n3. [Fight Club (1999)] - This thought-provoking film follows an insomniac office worker who forms an underground fight club as a means of escape and rebellion.\n4. [Eternal Sunshine of the Spotless Mind (2004)] - A unique romantic sci-fi flick that explores memory, love, and the consequences of erasing one's past.\n5. [The Grand Budapest Hotel (2014)] - A whimsical comedy-drama set in a fictional Eastern European hotel, filled with eccentric characters and an intriguing mu

In [322]:
response.choices[0].message.content

"Here are ten movie recommendations for you to watch:\n\n1. [Pulp Fiction (1994)] - Directed by Quentin Tarantino, this iconic crime film intertwines various characters and their stories in a non-linear narrative.\n2. [The Shawshank Redemption (1994)] - A gripping drama about a man who, despite being wrongfully convicted, finds hope and redemption during his time in prison.\n3. [Fight Club (1999)] - This thought-provoking film follows an insomniac office worker who forms an underground fight club as a means of escape and rebellion.\n4. [Eternal Sunshine of the Spotless Mind (2004)] - A unique romantic sci-fi flick that explores memory, love, and the consequences of erasing one's past.\n5. [The Grand Budapest Hotel (2014)] - A whimsical comedy-drama set in a fictional Eastern European hotel, filled with eccentric characters and an intriguing murder mystery.\n6. [Whiplash (2014)] - A thrilling drama about a young jazz drummer who enters a cutthroat music conservatory and forms a complex 

In [358]:
new_line = '\n'
for each in ten_closest_users_by_rating_distance.iloc[:1].index:
    Input = f"""I am user {each}.
The movies I have seen are are:
{user_movies[each]}.
Please recommend ten movies for me to watch that I have not seen and provide brackets around the movies and their years so I can easily parse them."""

In [370]:
new_line = '\n'
for each in ten_closest_users_by_rating_distance.iloc[:1].index:
    Input = f"""I am user {each}.
The movies I have seen are are:
{", ".join(user_movies[each])}.
The users who are most like me are {ten_closest_users_by_rating_distance[each]}.
The top movies for each of these users are:
{new_line.join(pd.Series(users_top_movies.loc[ten_closest_users_by_rating_distance[each]].index).apply(lambda x: ": ".join([str(x), ", ".join(users_top_movies.loc[x])])).to_list())}.
Please recommend ten movies for me to watch that I have not seen and provide brackets around the movies and their years so I can easily parse them."""

In [357]:
new_line = '\n'
for each in ten_closest_users_by_rating_distance.iloc[:1].index:
    Input = f"""I am user {each}.
The most recent ten movies I have seen are are:
{", ".join(user_movies[each][-10:])}.
Please recommend ten movies for me to watch that I have not seen and provide brackets around the movies and their years so I can easily parse them."""

In [ ]:
new_line = '\n'
for each in ten_closest_users_by_rating_distance.iloc[:1].index:
    Input = f"""I am user {each}.
The most recent ten movies I have seen are are:
{", ".join(user_movies[each][-10:])}.
The users who are most like me are {ten_closest_users_by_rating_distance[each]}.
The top movies for each of these users are:
{new_line.join(pd.Series(users_top_movies.loc[ten_closest_users_by_rating_distance[each]].index).apply(lambda x: ": ".join([str(x), ", ".join(users_top_movies.loc[x])])).to_list())}.
Please recommend ten movies for me to watch that I have not seen and provide brackets around the movies and their years so I can easily parse them."""

In [373]:
recommendations = [
    each.split("]")[0]
    for each in ([
        each
        for each in response.choices[0].message.content.split("[")
    ][1:])
]

In [337]:
test = pd.read_csv('../data/test.csv')

In [338]:
user_movies_test = test.groupby('user_id').apply(lambda x: x['movie_title'].tolist())

In [377]:
len(user_movies_test[1])

19

In [374]:
recommendations

['Casablanca (1942)',
 'The Shawshank Redemption (1994)',
 'The Godfather: Part I (1972)',
 'The Dark Knight (2008)',
 'Fight Club (1999)',
 'Inception (2010)',
 'The Matrix (1999)',
 'The Lion King (1994)',
 'Good Will Hunting (1997)',
 'Eternal Sunshine of the Spotless Mind (2004)']

In [375]:
set(user_movies_test[1]).intersection(recommendations)

{'Good Will Hunting (1997)'}

In [438]:
hits = pd.concat((user_movies_test, recommendations), axis=1).dropna().apply(lambda x: len(set(x.iloc[0]).intersection(x.iloc[1])), axis = 1)

In [439]:
hits

user_id
1     0
13    0
14    0
23    0
30    1
dtype: int64

In [430]:
recommendations

user_id
1     [Star Wars: Episode IV - A New Hope (1977), Th...
5     [Blade Runner, Pulp Fiction, The Godfather, Th...
6     [The Shawshank Redemption (1994), Fight Club (...
8     [The Shawshank Redemption (1994), The Godfathe...
10    [The Godfather: Part II (1974), Fight Club (19...
Name: user_id, dtype: object

In [431]:
user_movies_test

user_id
1      [Good Will Hunting (1997), Starship Troopers (...
2      [English Patient, The (1996), Contact (1997), ...
3      [Air Force One (1997), Game, The (1997), L.A. ...
4      [Contact (1997), Air Force One (1997), Scream ...
7      [Full Monty, The (1997), Volcano (1997), I Kno...
                             ...                        
936    [Good Will Hunting (1997), Contact (1997), Tit...
938    [Titanic (1997), Air Force One (1997), Contact...
940    [Sum of Us, The (1994), Some Like It Hot (1959...
942    [Apt Pupil (1998), Amistad (1997), Full Monty,...
943    [Taxi Driver (1976), Monty Python and the Holy...
Length: 455, dtype: object